In [115]:
links = []
with open('./links/2.txt', 'r') as file:
    # Read and process each line
    for line in file:
        splitted = line.split(" ")
        n = int(splitted[0])
        nprime = int(splitted[1])
        links.append((n, nprime))


In [116]:
counter = 0
titles = {}
with open('./titles/2.txt', 'r') as file:
    # Read and process each line
    for line in file:
        counter+=1
        titles[counter] = line
degrees_in = {}
degrees_out = {}

for nnprime in links:
    n = nnprime[0]
    nprime = nnprime[1]

    if(n in degrees_out.keys()):
        degrees_out[n].append(nprime)
    else:
        degrees_out[n] = [nprime]

    if(nprime in degrees_in.keys()):
        degrees_in[nprime].append(n)
    else:
        degrees_in[nprime] = [n]

In [117]:
num_degrees_in = {}
num_degrees_out = {}
for i in degrees_in:
    num_degrees_in[i] = len(degrees_in[i])
for i in degrees_out:
    num_degrees_out[i] = len(degrees_out[i])
sorted_degrees_in = dict(sorted(num_degrees_in.items(), key=lambda x: x[1], reverse=True))
sorted_degrees_out = dict(sorted(num_degrees_out.items(), key=lambda x: x[1], reverse=True))

list1 = []
list2 = []

node_ids_list1 = []
node_ids_list2 = []

normalized_in_degree = {}
normalized_out_degree = {}


for i in sorted_degrees_in:
    node_ids_list1.append(i)
    normalized_in_degree[i] = num_degrees_in[i]/(len(links))
    if(i in sorted_degrees_out.keys()):
        list1.append((degrees_in[i], degrees_out[i]))
    else:
        list1.append((degrees_in[i], []))

for i in sorted_degrees_out:
    normalized_out_degree[i] = num_degrees_out[i]/(len(links))
    node_ids_list2.append(i)
    if(i in sorted_degrees_in.keys()):
        list2.append((degrees_in[i], degrees_out[i])) 
    else:
        list2.append(([], degrees_out[i]))

In [118]:
print("IN DEGREES")
for node_id in node_ids_list1[:5]:
    title = titles[node_id]
    print(title, round(normalized_in_degree[node_id],6), round(normalized_out_degree[node_id],6))

print("OUT DEGREES")
for node_id in node_ids_list2[:5]:
    title = titles[node_id]
    print(title, round(normalized_out_degree[node_id],6), round(normalized_in_degree[node_id],6))

IN DEGREES
2007
 0.007685 0.000254
2008
 0.006608 1.2e-05
United_States
 0.006596 1.2e-05
United_Kingdom
 0.00541 0.000218
France
 0.005156 0.001476
OUT DEGREES
.cf
 0.003231 4.8e-05
Book_of_Optics
 0.003171 0.000242
Bengali_language
 0.002844 0.000351
Dumfries,_VA
 0.002759 1.2e-05
Ahmad_Zarruq
 0.002638 0.001404


## TASK 2

In [119]:
def normalize(dictionary):
    tot = sum(dictionary.values())
    for key in dictionary.keys():
        dictionary[key] = dictionary[key]/tot
    return dictionary

def get_new_hub_score(degrees_out, auths):
    #iterate over all hubs to update
    new_hub = {}
    for hub_node in degrees_out.keys():
        #gets all the nodes pointing to hub node
        new_hub_score = 0
        for auth_node in degrees_out[hub_node]:
            new_hub_score += auths[auth_node]
        new_hub[hub_node] = new_hub_score
    return normalize(new_hub)

def get_new_auth_score(degrees_in, hubs):
    new_auth = {}
    for auth_node in degrees_in.keys():
        new_auth_score = 0
        for out_node in degrees_in[auth_node]:
            new_auth_score += hubs[out_node]
        new_auth[auth_node] = new_auth_score
    return normalize(new_auth)

hubs = normalized_out_degree
auths = normalized_in_degree

for i in range(100):
    hubs = get_new_hub_score(degrees_out, auths)
    auths = get_new_auth_score(degrees_in, hubs)


sorted_on_hub_score = dict(sorted(hubs.items(), key=lambda x: x[1], reverse=True))
sorted_on_auth_score = dict(sorted(auths.items(), key=lambda x: x[1], reverse=True))

top5_hubs = list(sorted_on_hub_score.keys())[:5]
top5_auths = list(sorted_on_auth_score.keys())[:5]
print("HUB")
for i in top5_hubs:
    if(i not in sorted_on_auth_score.keys()):
        print(titles[i], round(sorted_on_hub_score[i],6), 0.00)
    else:
        print(titles[i], round(sorted_on_hub_score[i],6), round(sorted_on_auth_score[i],6))

print("AUTH")
for i in top5_auths:
    if(i not in sorted_on_hub_score.keys()):
        print(titles[i], round(sorted_on_auth_score[i],6), 0.00)
    else:
        print(titles[i], round(sorted_on_auth_score[i],6), round(sorted_on_hub_score[i],6))


HUB
.cf
 0.006991 0.0
Book_of_Optics
 0.006978 7.8e-05
Bengali_language
 0.006846 0.00014
Erich_Schmidt_(archaeologist)
 0.006811 1e-06
1999_NATO_bombing_of_the_Federal_Republic_of_Yugoslavia
 0.006803 4e-06
AUTH
Shams_Tabrizi
 0.006591 0.000277
Mahmud_Shabistari
 0.00658 0.0003
Al-Ghazali
 0.006541 0.0
Shahab_al-Din_Suhrawardi
 0.006472 0.0
Farid_al-Din_Attar
 0.00646 0.0



## TASK 3

In [120]:
import math


def get_eigenvector(degrees_in, centrality_in):
    for node in degrees_in.keys():
        centrality = 0
        for node_in in degrees_in[node]:
            if(node_in in centrality_in.keys()):
                centrality += centrality_in[node_in]
        centrality_in[i] = centrality
    return centrality_in

centrality_in = num_degrees_in
        
for i in range(200):
    centrality_in = get_eigenvector(degrees_in, centrality_in)

centrality_in = normalize(centrality_in)
sorted_on_vector_score = dict(sorted(centrality_in.items(), key=lambda x: x[1], reverse=True))

top5_centrality = list(sorted_on_vector_score.keys())[:5]
print("HUB")
for i in top5_centrality:
    print(titles[i], round(sorted_on_vector_score[i],6), 0.00)

HUB
United_States
 0.007305 0.0
United_Kingdom
 0.005992 0.0
France
 0.00571 0.0
Iran
 0.005228 0.0
Wikimedia_Commons
 0.004946 0.0


In [121]:
import numpy as np
import scipy.sparse as sp
cols, rows = zip(*[(i - 1, j - 1) for i, j in links])
data = np.ones(len(rows), dtype=int)

# Create the sparse matrix using coo_matrix
size = len(list(titles.keys()))
sparse_matrix = sp.coo_matrix((data, (rows, cols)), shape=(size, size), dtype=np.float64)

# Calculate the eigenvector centrality
eigen_value, eigen_vector = sp.linalg.eigs(sparse_matrix,k=1, which="LR")  # Use transpose for right eigenvector
eigen_vector = np.abs(eigen_vector.flatten())
eigen_vector /= eigen_vector.sum()

sorted_indices = np.argsort(eigen_vector)[::-1]

# Get the values and indices of the first 5 elements
top5_eigen = eigen_vector[sorted_indices[:5]]
top5_eigen_indices = sorted_indices[:5]
for i in top5_eigen_indices:
    print(titles[i+1], round(eigen_vector[i],6))
    print(i)

2007
 0.006334
71
United_States
 0.006312
2770
2008
 0.005851
73
France
 0.005535
946
United_Kingdom
 0.005324
2765


## TASK 4


In [122]:
eigen_value = np.real(eigen_value)

alpha = (0.85*(1/abs(eigen_value)))
num_nodes = len(titles.keys())

I = np.identity(num_nodes)
u = np.ones(num_nodes)

A = sparse_matrix.toarray()

te = (I-alpha*A)

inverted = np.linalg.inv(te)

centrality_vector = (1/num_nodes)*inverted@u
centrality_vector /= centrality_vector.sum()

sorted_katz = np.argsort(centrality_vector)[::-1]

# Get the values and indices of the first 5 elements
katz_top5 = centrality_vector[sorted_katz[:5]]
top5_eigen_indices = sorted_katz[:5]
for i in sorted_katz[:5]:
    print(titles[i+1], round(centrality_vector[i],6))


2007
 0.005583
United_States
 0.005411
2008
 0.005085
France
 0.004673
United_Kingdom
 0.004544


## TASK 5

In [211]:
cop = num_degrees_out.copy()
kout = []
for i in range(1, num_nodes+1):
    if(i not in cop.keys()):
        kout.append(0)
    else:
        kout.append(1/cop[i])
H = A * kout
alpha = 0.85
u = np.ones(num_nodes)
I = np.identity(num_nodes)
aH = alpha*H
inverted = np.linalg.inv(I-aH)
closed_gp = ((1-alpha)/num_nodes)*inverted@u
closed_gp /= closed_gp.sum()

sorted_closed_gp = np.argsort(closed_gp)[::-1]

for i in sorted_closed_gp[:5]:
    print(titles[i+1], round(closed_gp[i],6))

Portugal
 0.012437
United_States
 0.012286
2007
 0.010234
2008
 0.00812
Wikimedia_Commons
 0.00697


## TASK 6


In [137]:

uut = np.ones((num_nodes, num_nodes))

# Calculate G
G = alpha * H + ((1 - alpha) / num_nodes) * uut
print(G.shape)

dict_values([1, 267, 1, 212, 1, 124, 131, 10, 131, 68, 23, 22, 109, 104, 1, 154, 71, 75, 81, 114, 108, 9, 139, 121, 1, 121, 120, 117, 188, 7, 1, 6, 54, 52, 30, 1, 68, 12, 107, 1, 74, 69, 70, 127, 73, 88, 110, 97, 11, 8, 1, 183, 13, 34, 6, 34, 114, 10, 20, 19, 4, 91, 1, 210, 115, 1, 130, 1, 76, 1, 21, 93, 1, 157, 159, 8, 22, 1, 4, 1, 8, 1, 19, 9, 1, 29, 8, 28, 24, 29, 1, 38, 9, 2, 10, 11, 3, 209, 22, 1, 18, 2, 10, 117, 76, 74, 71, 73, 72, 75, 70, 5, 65, 12, 25, 87, 12, 1, 8, 2, 5, 151, 85, 75, 5, 1, 3, 97, 19, 1, 19, 13, 12, 11, 6, 12, 5, 4, 2, 1, 1, 3, 18, 94, 1, 93, 1, 93, 218, 124, 128, 18, 1, 15, 22, 1, 13, 4, 7, 19, 14, 5, 5, 29, 1, 20, 7, 4, 5, 6, 1, 51, 13, 17, 13, 17, 2, 8, 41, 12, 7, 15, 5, 92, 93, 17, 4, 104, 17, 1, 5, 22, 17, 139, 1, 28, 134, 11, 1, 3, 106, 34, 106, 6, 8, 5, 4, 12, 6, 11, 76, 90, 1, 207, 1, 21, 8, 85, 95, 184, 25, 27, 32, 12, 118, 1, 54, 1, 70, 18, 27, 18, 5, 113, 1, 88, 1, 30, 120, 174, 9, 8, 153, 116, 4, 146, 11, 14, 130, 132, 119, 177, 20, 2, 51, 82, 28, 9

In [143]:
%history

links = []
with open('./links/2.txt', 'r') as file:
    # Read and process each line
    for line in file:
        splitted = line.split(" ")
        n = int(splitted[0])
        nprime = int(splitted[1])
        links.append((n, nprime))
%history
counter = 0
titles = {}
with open('./titles/2.txt', 'r') as file:
    # Read and process each line
    for line in file:
        counter+=1
        titles[counter] = line
degrees_in = {}
degrees_out = {}

for nnprime in links:
    n = nnprime[0]
    nprime = nnprime[1]

    if(n in degrees_out.keys()):
        degrees_out[n].append(nprime)
    else:
        degrees_out[n] = [nprime]

    if(nprime in degrees_in.keys()):
        degrees_in[nprime].append(n)
    else:
        degrees_in[nprime] = [n]
num_degrees_in = {}
num_degrees_out = {}
for i in degrees_in:
    num_degrees_in[i] = len(degrees_in[i])
for i in degrees_out:
    num_degrees_out[i] = len(degrees_out[i])
sorted_degrees_in = dict(sorted(num_degrees_in.items(), key=lambda x: x[1], 

In [138]:
eigenvalues, eigenvectors = np.linalg.eig(A)
print(np.where(eigenvalues == 1))

(array([], dtype=int64),)
